# Deteksi Berita Palsu Bidang Kesehatan dengan Snorkel dan Perceptron

Proyek ini menggunakan pendekatan **Rule-Based Labeling** dengan **Snorkel** untuk memberi label data berita kesehatan secara otomatis, kemudian melatih model **Perceptron** untuk mendeteksi berita palsu.

## 1. Instalasi Library yang Diperlukan
Jalankan perintah berikut di terminal atau cell ini untuk menginstal library yang dibutuhkan:

In [18]:
!pip install snorkel pandas scikit-learn numpy nltk --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 88.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 89.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.2.3 which is incompatible.
tensorflow 2.18.0 requ

## 2. Import Library

In [19]:
import pandas as pd
import numpy as np
from snorkel.labeling import labeling_function, PandasLFApplier
from snorkel.labeling.model import LabelModel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

## 3. Memuat dan Preprocessing Data
Data yang diberikan adalah daftar judul berita dari Kompas.com. Kita akan memuatnya ke dalam DataFrame dan melakukan preprocessing sederhana.

In [34]:
# Konstanta Label
ABSTAIN = -1
FAKE = 0
TRUE = 1

# Buat DataFrame
df = pd.DataFrame(pd.read_csv('https://docs.google.com/spreadsheets/d/11TJNGsOvSxns-Vu2dLtylK_7D6e_5zrl/export?format=csv', on_bad_lines='skip'))

# Preprocessing teks
stop_words = set(stopwords.words('indonesian'))
def preprocess_text(text):
    text = text.lower()  # Huruf kecil
    text = re.sub(r'[^a-z\s]', '', text)  # Hapus tanda baca dan angka
    text = ' '.join([word for word in text.split() if word not in stop_words])  # Hapus stop words
    return text

df['Judul Berita'] = df['Judul Berita'].apply(preprocess_text)
print("Data setelah preprocessing:")
print(df[['No', 'Judul Berita']].head())

Data setelah preprocessing:
   No                                       Judul Berita
0   1  tragedi penghapusan mandatory spending uu kese...
1   2                pindah faskes bpjs kesehatan online
2   3  uu kesehatan terbaru str dokter perawat berlak...
3   4   uu kesehatan hapus anggaran wajib minimal bidang
4   5  uu kesehatan bolehkan dokter asing berpraktik ...


## 4. Definisi Labeling Functions dengan Snorkel
Kita akan membuat aturan sederhana untuk mendeteksi potensi berita palsu berdasarkan kata kunci sensasional atau kontradiksi.

In [14]:
# Konstanta untuk label
ABSTAIN = -1
FAKE = 0
REAL = 1

# Labeling Function 1: Berita dengan kata sensasional dianggap potensi palsu
@labeling_function()
def sensational_keywords(x):
    keywords = ['tragedi', 'ramai', 'kontroversi', 'demo', 'tolak']
    return FAKE if any(keyword in x['Judul Berita'] for keyword in keywords) else ABSTAIN

# Labeling Function 2: Berita dengan panduan praktis dianggap asli
@labeling_function()
def practical_guide(x):
    keywords = ['cara', 'langkah', 'daftar', 'tips']
    return REAL if any(keyword in x['Judul Berita'] for keyword in keywords) else ABSTAIN

# Labeling Function 3: Berita dengan sumber resmi dianggap asli
@labeling_function()
def official_source(x):
    return REAL if 'kemenkes' in x['Judul Berita'] or 'jokowi' in x['Judul Berita'] else ABSTAIN

# Daftar semua labeling functions
lfs = [sensational_keywords, practical_guide, official_source]

## 5. Menerapkan Labeling Functions dengan Snorkel

In [15]:
# Terapkan labeling functions ke data
applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df)

# Gunakan LabelModel untuk menggabungkan hasil labeling functions
label_model = LabelModel(cardinality=2, verbose=True)
label_model.fit(L_train, n_epochs=500, log_freq=100, seed=123)

# Prediksi label probabilistik
df['label'] = label_model.predict(L_train)
df.head()

100%|██████████| 500/500 [00:00<00:00, 759.53epoch/s]


,No,Judul Berita,Sumber,Tanggal,Kategori,label
0,1,tragedi penghapusan mandatory spending dalam u...,Kompas.com,2023-08-13,Kesehatan,0
1,2,cara pindah faskes bpjs kesehatan online,Kompas.com,2024-07-13,Kesehatan,1
2,3,uu kesehatan terbaru str dokter dan perawat be...,Kompas.com,2023-07-11,Kesehatan,-1
3,4,uu kesehatan baru hapus anggaran wajib minimal...,Kompas.com,2023-07-12,Kesehatan,-1
4,5,uu kesehatan bolehkan dokter asing berpraktik ...,Kompas.com,2023-07-15,Kesehatan,1


## 6. Ekstraksi Fitur dan Pelatihan Model Perceptron
Kita akan mengubah teks menjadi fitur numerik menggunakan TF-IDF, lalu melatih model Perceptron.

In [16]:
# Cell 6: Ekstraksi Fitur dan Pelatihan Model Perceptron
# Ekstraksi fitur dengan TF-IDF
vectorizer = TfidfVectorizer(max_features=500)
X = vectorizer.fit_transform(df['Judul Berita']).toarray()
y = df['label']

# Split data menjadi training dan testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Filter data yang memiliki label valid (bukan ABSTAIN)
mask_train = y_train != ABSTAIN
mask_test = y_test != ABSTAIN
X_train, y_train = X_train[mask_train], y_train[mask_train]
X_test, y_test = X_test[mask_test], y_test[mask_test]

# Latih model Perceptron
perceptron = Perceptron(max_iter=1000, tol=1e-3, random_state=42)
perceptron.fit(X_train, y_train)

# Prediksi pada data test
y_pred = perceptron.predict(X_test)

# Ubah label numerik menjadi "Fake" atau "True"
y_test_str = ['Fake' if label == FAKE else 'True' for label in y_test]
y_pred_str = ['Fake' if label == FAKE else 'True' for label in y_pred]

# Evaluasi
print('Accuracy:', accuracy_score(y_test, y_pred))
print('Classification Report:\n', classification_report(y_test_str, y_pred_str, target_names=['Fake', 'True']))

# Tampilkan hasil prediksi pada seluruh data
df['Prediksi'] = ['Fake' if label == FAKE else 'True' if label == REAL else 'Abstain' for label in label_model.predict(L_train)]
print("\nHasil Prediksi pada Data:")
print(df[['Judul Berita', 'Prediksi']])

Accuracy: 0.0
Classification Report:
               precision    recall  f1-score   support

        Fake       0.00      0.00      0.00       0.0
        True       0.00      0.00      0.00       1.0

    accuracy                           0.00       1.0
   macro avg       0.00      0.00      0.00       1.0
weighted avg       0.00      0.00      0.00       1.0


Hasil Prediksi pada Data:
                                        Judul Berita Prediksi
0  tragedi penghapusan mandatory spending dalam u...     Fake
1           cara pindah faskes bpjs kesehatan online     True
2  uu kesehatan terbaru str dokter dan perawat be...  Abstain
3  uu kesehatan baru hapus anggaran wajib minimal...  Abstain
4  uu kesehatan bolehkan dokter asing berpraktik ...     True


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

## 7. Contoh Prediksi pada Data Baru

In [17]:
# Cell 7: Contoh Prediksi pada Data Baru
new_titles = [
    'cara cepat sembuh dari flu dengan bpjs kesehatan',
    'tragedi besar dokter asing bunuh pasien di ri'
]
new_titles_cleaned = [preprocess_text(title) for title in new_titles]
X_new = vectorizer.transform(new_titles_cleaned).toarray()
predictions = perceptron.predict(X_new)

print("\nPrediksi pada Data Baru:")
for title, pred in zip(new_titles, predictions):
    label = 'Fake' if pred == FAKE else 'True'
    print(f'Judul: {title} -> Prediksi: {label}')


Prediksi pada Data Baru:
Judul: cara cepat sembuh dari flu dengan bpjs kesehatan -> Prediksi: Fake
Judul: tragedi besar dokter asing bunuh pasien di ri -> Prediksi: True
